In [2]:
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import math
from xhistogram.xarray import histogram
%load_ext autoreload
%autoreload 2
import pei.myfunctions as mf

In [3]:
# Specify root directory, experiment, filename format
rootdir = '/local/ss23/CESM2_LE/ATM/RCP85/WBT'
filename_2085 = '/b.e21.BSSP370cmip6.f09_g17.LE2-1231.*.clm2.h7.WBT.2085010100-2095010100.nc' 
paths_2085 = rootdir+filename_2085

# Load data from matching files
ds_2085 = xr.open_mfdataset(paths_2085,combine='nested',concat_dim='ensemble')['WBT']
mean_2085 = ds_2085.mean(dim='ensemble').mean(dim='time')

In [ ]:
filename_1980 = '/b.e21.BHISTcmip6.f09_g17.LE2-1231.*.clm2.h7.WBT.1980010100-1990010100.nc' 
paths_1980 = rootdir+filename_2085

# Load data from matching files
ds_1980 = xr.open_mfdataset(paths_1980,combine='nested',concat_dim='ensemble')['WBT']
mean_1980 = ds_1980.mean(dim='ensemble').mean(dim='time')

In [ ]:
# Plot global temperature
mf.contour_plot(mean_2085,region='Global',title='2085-2095 Decadal Mean WBT: Global',cmap='Reds')

In [ ]:
mean_diff = mean_2085-mean_1980
mf.contour_plot(mean_diff,region='Global',title='Difference in Decadal Mean Temperature: 2090s - 1980s, Global',cmap='OrRd')